In [ ]:
!nvidia-smi

Fri Apr  8 02:27:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 17.5 MB/s 
     |████████████████████████████████| 6.5 MB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import json
content={"username":"namlv1997","key":"5ecc995bb891689ee7f58417d65cbe37"}
with open('kaggle.json','w')as f:
  json.dump(content,f)

In [ ]:
!mv /content/kaggle.json "/root/.kaggle/kaggle.json"

In [ ]:
!kaggle competitions download -c fake-news

 65% 30.0M/46.5M [00:00<00:00, 117MB/s] 
100% 46.5M/46.5M [00:00<00:00, 125MB/s]


In [ ]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [1]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from sklearn.metrics import f1_score
from transformers import AutoConfig

In [2]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="distilbert-base-uncased"

In [13]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'

In [14]:
os.makedirs(model_path, exist_ok=True)

# Load fake news dataset from kaggle

In [6]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.drop(['id','title','author'],axis=1)
df=df.dropna(subset=['label', 'text'])
df

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0
20796,When the Green Bay Packers lost to the Washing...,0
20797,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Split raw dataset to train set and validation set

In [7]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
valid = df[~msk]

Check data whether it is balanced or not

In [8]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 8329 samples
label: 1	has 8366 samples


In [9]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 2058 samples
label: 1	has 2008 samples


In [10]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [11]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [15]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)


In [16]:
"""This function is from The Stanford Question Answering Dataset evaluate"""
def normalize(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  return white_space_fix(remove_articles(remove_punc(s))).strip()

In [17]:
train['text']=train['text'].map(normalize)

In [18]:
valid['text']=valid['text'].map(normalize)

In [19]:
train=train[train['text']!=""]
valid=valid[valid['text']!=""]

In [21]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [22]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [23]:
tokenizer = AutoTokenizer.from_pretrained(name)

In [24]:
max_length=512
return_tensors='tf'

In [25]:
x_train=tokenizer(train['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [26]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [27]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [28]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [29]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [30]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [31]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [32]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [33]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

In [34]:
epochs=5
lr=5e-5
num_warmup_steps=5000

In [35]:
optimizer, schedule = create_optimizer(
    init_lr=lr, num_warmup_steps=num_warmup_steps, num_train_steps=num_training_samples*epochs
)

In [36]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Load the pretrained model

In [37]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [38]:
model.base_model_prefix

'distilbert'

Define train_step function for training

For token_type_ids

In [39]:
train_step_signature1=[
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature1)
def train_step1(input_ids, attention_mask,token_type_ids,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,training = True)

    probs=tf.nn.softmax(outputs.logits,axis=1)
    preds=tf.argmax(probs,axis=1)
    loss = loss_fn(labels, probs)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss,preds

For not token_type_ids

In [40]:
train_step_signature2= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature2)
def train_step2(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = True)

    probs=tf.nn.softmax(outputs.logits,axis=1)
    preds=tf.argmax(probs,axis=1)
    loss = loss_fn(labels, probs)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss,preds

Defind test_step function for validation

In [41]:
def test_step(input_ids, attention_mask,token_type_ids,labels):
  
  if token_type_ids is None:
    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = False)
  else:
    outputs= model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,training = False)

  probs=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(probs,axis=1)
  loss = loss_fn(labels, probs)

  return loss,preds

In [42]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Initializing the new checkpoint


In [43]:
threshold=1e-6

Train

In [44]:
def run_step(input_ids,attention_mask,token_type_ids,labels,session='train'):
  if session=='train':
    if model.base_model_prefix in ['distilbert']:
      return train_step2(input_ids,attention_mask,labels)
    else:
      return train_step1(input_ids,attention_mask,token_type_ids,labels)
  else:
    return test_step(input_ids,attention_mask,token_type_ids,labels)


In [45]:
def step(dataset,session='train'):
  
  loop = tqdm(dataset)
  loss=0
  y_true=[]
  y_pred=[]
  num_batches=len(dataset)
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask'] if 'attention_mask' in batch else None
    token_type_ids=batch['token_type_ids'] if 'token_type_ids' in batch else None
    labels=batch['labels']

    _loss,preds=run_step(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels,session=session)

    y_true.append(batch['labels'])
    y_pred.append(preds)
    _loss=_loss.numpy()
    loss+=loss
    num_batches+=1
    loop.set_postfix({'train_loss_per_batch':_loss})
    loop.update()

  y_true=tf.concat(y_true,axis=0)
  y_pred=tf.concat(y_pred,axis=0)
  acc=f1_score(y_true,y_pred)
  loss=loss/num_batches
  return acc,loss

In [ ]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_acc,train_loss=step(train_dataset)

  """Validation"""
  valid_acc,valid_loss=step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} train_acc :{train_acc} valid_loss: {valid_loss} valid_acc :{valid_acc}')
  if tf.abs(min_loss-valid_loss)>threshold and min_loss>valid_loss:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')
    min_loss=valid_loss

  0%|          | 0/2079 [00:00<?, ?it/s]

# Inferencing

Load test set

In [6]:
test=pd.read_csv("/content/test.csv")

In [7]:
submit=pd.read_csv("/content/submit.csv")
labels=tf.convert_to_tensor(submit['label'],dtype=tf.float32)

In [8]:
test['text']=test['text'].astype(str).map(normalize)

In [9]:
samples=test['text'].tolist()

In [10]:
num_labels=len(submit['label'].unique())
num_labels

2

Load model

In [11]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [12]:
model=TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.num_labels

2

In [14]:
max_length=512
return_tensors='tf'
tokenizer = AutoTokenizer.from_pretrained(name)

Load the weights

In [15]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Loading the latest checkpoint from /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-1


In [16]:
y_pred=[]
loop = tqdm(range(len(samples)))
for idx,sample in enumerate(samples):
  inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)
  outputs=model(**inputs,training=False)
  prob=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(prob,axis=1)
  y_pred.append(preds)
  loop.update()


100%|██████████| 5200/5200 [15:35<00:00,  5.74it/s]

In [17]:
y_pred=tf.convert_to_tensor(y_pred)

In [18]:
y_pred=tf.reshape(y_pred,shape=(y_pred.shape[0],))
y_pred

<tf.Tensor: shape=(5200,), dtype=int64, numpy=array([0, 1, 1, ..., 0, 1, 0])>

In [19]:
labels

<tf.Tensor: shape=(5200,), dtype=float32, numpy=array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)>

In [20]:
f1_score(labels,y_pred,average='micro')

0.635576923076923